In [11]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score
from tqdm import tqdm
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.svm import SVC
from xgboost import XGBClassifier
import pickle
import tensorflow as tf
import math
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier

In [2]:
""" One v.s. All binary classification """

' One v.s. All binary classification '

In [3]:
df = pd.read_csv("../Data_Scrape/Dataset/updated_dataset.csv")

In [4]:
df.head()

,sp_song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,lda_topic_11,lda_topic_12,lda_topic_13,lda_topic_14,lda_topic_15,lda_topic_16,lda_topic_17,lda_topic_18,lda_topic_19,Playlists
0,6edQfeOlqbGteYixpJl3Sm,0.857,0.602,10,-8.311,1,0.0290,0.489000,0.02440,0.3170,...,0.0,0.022993,0.000000,0.122078,0.0,0.000000,0.000000,0.081108,0.386042,[165]
1,5Oe7wHPL4hdEXeF4AOayCi,0.322,0.990,8,-1.785,1,0.1710,0.000044,0.41700,0.0366,...,0.0,0.000000,0.163142,0.022999,0.0,0.144614,0.187207,0.000000,0.000000,[79]
2,6i1uWZYWabNHq2wQnoca58,0.666,0.884,9,-5.243,0,0.0429,0.506000,0.00612,0.0408,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.582131,[15]
3,4BzBtS6PBreni5hNPo2hos,0.609,0.578,9,-7.081,1,0.0414,0.296000,0.00000,0.1500,...,0.0,0.000000,0.511716,0.084905,0.0,0.000000,0.000000,0.115004,0.108140,[168]
4,0GvhHQbWSnGltjl0je61dI,0.699,0.514,4,-12.610,1,0.0315,0.587000,0.02900,0.2200,...,0.0,0.000000,0.000000,0.401954,0.0,0.000000,0.000000,0.000000,0.000000,"[30, 133]"


In [ ]:
playlists_multi = [132, 123,19,46,60,68,79,125]  #132, 123,19,46,60,68,79,125
# playlists_multi = [i for i in range(1,169,1)]  #132, 123,19,46,60,68,79,125

dict_playlists_to_songs = {}
data = np.array(df)
Dataset = []
for j in playlists_multi:
    l0 = []
    for i in df.index:
        l = eval(df["Playlists"][i])
        if(j in l):
            l.remove(j)
            flag = 0
            for k in playlists_multi:
                    if(k!=j and k in l):
                        flag = 1
            if(flag!=1):
                l0.append(df["sp_song_id"][i])
                Dataset.append(data[i])
                
    dict_playlists_to_songs[j] = l0
#     print(len(l0))
print(len(dict_playlists_to_songs))
Dataset = np.array(Dataset)
print(Dataset.shape)
np.random.shuffle(Dataset)
print(Dataset[0])



In [ ]:
X = Dataset[:,1:-1]
Y = Dataset[:,-1]
# print(Y)
for i in range(len(Y)):
    Y[i] = eval(Y[i])
    for j in playlists_multi:
        if(j in Y[i]):
            Y[i] = j
            break
    
print(X.shape)
print(Y.shape)



In [ ]:
X = np.array(X)
y = np.array(Y)
y = Y.reshape(-1, 1)
standardScalar = StandardScaler()
X = standardScalar.fit_transform(X)
print(X.shape, y.shape)
y = np.squeeze(y)
# print(X)
# print(y)

In [ ]:
y = y.astype("str")
# print(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tsne_em = TSNE(n_components=2, perplexity=30.0, n_iter=1000, verbose=1).fit_transform(X_train)
print(tsne_em)

In [ ]:
df_tsne= pd.DataFrame(columns = ["x", "y", "label"] )
df_tsne["x"] = tsne_em[:,0]
df_tsne["y"] = tsne_em[:,1]
df_tsne["label"] = y_train

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize = (20,20))
# plt.scatter(tsne_em[:,0], tsne_em[:,1],color = "green", hue = y_train);
sns.scatterplot(x = df_tsne["x"], y = df_tsne["y"], hue = df_tsne["label"],  palette = sns.color_palette("husl", len(set(y_train))))
plt.show()

In [ ]:

logistic = LogisticRegression(solver = "saga", multi_class = "multinomial")

logistic.fit(X_train, y_train)

test_score = logistic.score(X_test, y_test)

y_pred = logistic.predict(X_test)
pres_score = precision_score(y_test, y_pred, average = "weighted")

f1= f1_score(y_test, y_pred, average = "weighted")
recall_Score = recall_score(y_test, y_pred, average = "weighted")
print(test_score, pres_score, f1, recall_Score)

In [ ]:
""" One vs One Binary classification"""

In [7]:
df = pd.read_csv("../Data_Scrape/Dataset/updated_dataset.csv")

In [8]:
df.head()

,sp_song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,lda_topic_11,lda_topic_12,lda_topic_13,lda_topic_14,lda_topic_15,lda_topic_16,lda_topic_17,lda_topic_18,lda_topic_19,Playlists
0,6edQfeOlqbGteYixpJl3Sm,0.857,0.602,10,-8.311,1,0.0290,0.489000,0.02440,0.3170,...,0.0,0.022993,0.000000,0.122078,0.0,0.000000,0.000000,0.081108,0.386042,[165]
1,5Oe7wHPL4hdEXeF4AOayCi,0.322,0.990,8,-1.785,1,0.1710,0.000044,0.41700,0.0366,...,0.0,0.000000,0.163142,0.022999,0.0,0.144614,0.187207,0.000000,0.000000,[79]
2,6i1uWZYWabNHq2wQnoca58,0.666,0.884,9,-5.243,0,0.0429,0.506000,0.00612,0.0408,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.582131,[15]
3,4BzBtS6PBreni5hNPo2hos,0.609,0.578,9,-7.081,1,0.0414,0.296000,0.00000,0.1500,...,0.0,0.000000,0.511716,0.084905,0.0,0.000000,0.000000,0.115004,0.108140,[168]
4,0GvhHQbWSnGltjl0je61dI,0.699,0.514,4,-12.610,1,0.0315,0.587000,0.02900,0.2200,...,0.0,0.000000,0.000000,0.401954,0.0,0.000000,0.000000,0.000000,0.000000,"[30, 133]"


In [9]:
""" SVM - Linear Classification  """

' SVM - Linear Classification  '

In [90]:
def run_model(model_type = "LR", neighbours = 5):
    playlists = []
    result_history = {}

    np.random.seed(123)

    for playlist in tqdm(range(0,169,1), desc = "Progress : ", position = 0, leave = True):
        X = []
        y = []
        list_of_is = []
        for i in df.index:
            a = list(df.iloc[i])[1:-1]
            if playlist in eval(df['Playlists'][i]):
                X.append(a)
                y.append(1)
            else:
                _ = np.random.randint(0, 26)
                if _ == 0:
                    X.append(a)
                    y.append(0)



        X = np.array(X)
        y = np.array(y)



        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10, stratify = y)

        standardScalar = StandardScaler()
        X_train = standardScalar.fit_transform(X_train)
        X_test = standardScalar.transform(X_test)
        
        
        if(model_type == "LR" ):
            model = LogisticRegression()
            
        elif(model_type == "SVM"):
            model = SVC(kernel = 'linear')
        
        elif (model_type == "DT"):
            model = DecisionTreeClassifier(criterion = 'entropy', max_depth =  20)
            
        elif (model_type == "RF"):
            model = RandomForestClassifier(criterion = 'entropy')
            
        
        elif(model_type == "XGB"):
            
            param = {
                "learning_rate" : 0.1,
                "n_estimators" : 500,
                "max_depth" : 20,
                "min_child_weight" : 1,
                "gamma" : 0.1,
                "subsample": 0.9,
                "colsample_bytree" : 0.9,
                "objective" : 'binary:logistic',
                "nthread" : 4,
                "scale_pos_weight" :  1,
                "seed" : 27
            }
            
            model = XGBClassifier(**param)
        
        elif(model_type == "KNN"):
            model = KNeighborsClassifier(n_neighbors=neighbours)
            
        elif(model_type == "ANN"):
            model = Sequential([Dense(units = 24, input_shape = (34, ), activation = 'relu'),
            Dense(units = 12, activation = 'relu'),
            Dense(units = 4, activation = 'relu'),       
            Dense(units = 1, activation = 'sigmoid')
            ])
            model.compile(optimizer = tf.keras.optimizers.Adam(lr = 0.05), loss = 'binary_crossentropy', metrics = ['accuracy'])
            model.summary()
            
        


#         xgb = XGBClassifier(**param)    
        
        if model_type != 'ANN':
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            
        else:
            model.fit(X_train, y_train, validation_data = (X_test, y_test), batch_size = 64, epochs = 20)
            y_pred = model.predict_classes(X_test)
            

        pres_score = precision_score(y_test, y_pred, average = "macro")

        f1= f1_score(y_test, y_pred, average = "macro")
        recall_Score = recall_score(y_test, y_pred, average = "macro")

        result_history[playlist] = {"Precision Score " : pres_score, "Recall Score " : recall_Score, "F1 Score " : f1 }


    print(result_history)
    return result_history

In [ ]:
# f = open("binary_svc_linear_result", "wb")
# pickle.dump(result_history, f)
# f.close()

In [ ]:
f = open("binary_svc_linear_result", "rb")
l = pickle.load(f)
f.close()

In [ ]:
l = {k: v for k, v in sorted(l.items(), key=lambda item: item[1]["F1 Score "], reverse = True)}

In [ ]:
# Top 20 playlists 
for key in list(l.keys())[:20]:
    print(key," = ",  l[key])

In [ ]:
""" Logistic Regression Result"""

In [ ]:
result_history = run_model(model_type = "LR")

In [ ]:
# f = open("binary_linear_regression_result", "wb")
# pickle.dump(result_history, f)
# f.close()

In [ ]:
f = open("binary_linear_regression_result", "rb")
l = pickle.load(f)
f.close()

In [ ]:
l = {k: v for k, v in sorted(l.items(), key=lambda item: item[1]["F1 Score "], reverse = True)}

In [ ]:
# Top 20 playlists 
for key in list(l.keys())[:20]:
    print(key," = ",  l[key])

In [ ]:
""" Decision Tree Result"""

In [ ]:
result_history = run_model(model_type = "DT")

In [ ]:
# f = open("binary_decision_tree_result", "wb")
# pickle.dump(result_history, f)
# f.close()

In [ ]:
f = open("binary_decision_tree_result", "rb")
l = pickle.load(f)
f.close()

In [ ]:
l = {k: v for k, v in sorted(l.items(), key=lambda item: item[1]["F1 Score "], reverse = True)}

In [ ]:
# Top 20 playlists 
for key in list(l.keys())[:20]:
    print(key," = ",  l[key])

In [ ]:
""" Random Forest Result"""

In [ ]:
result_history = run_model(model_type = "RF")

In [ ]:
# f = open("binary_random_forest_result", "wb")
# pickle.dump(result_history, f)
# f.close()

In [ ]:
f = open("binary_random_forest_result", "rb")
l = pickle.load(f)
f.close()

In [ ]:
l = {k: v for k, v in sorted(l.items(), key=lambda item: item[1]["F1 Score "], reverse = True)}

In [ ]:
# Top 20 playlists 
for key in list(l.keys())[:20]:
    print(key," = ",  l[key])

In [ ]:
""" XG Boost Result"""

In [ ]:
result_history = run_model(model_type = "XGB")

In [ ]:
# f = open("binary_xgb_result", "wb")
# pickle.dump(result_history, f)
# f.close()

In [ ]:
f = open("binary_xgb_result", "rb")
l = pickle.load(f)
f.close()

In [ ]:
l = {k: v for k, v in sorted(l.items(), key=lambda item: item[1]["F1 Score "], reverse = True)}

In [ ]:
# Top 20 playlists 
for key in list(l.keys())[:20]:
    print(key," = ",  l[key])

KNN Classification

In [52]:
n_neighbours = 13

In [53]:
result_history = run_model(model_type = "KNN", neighbours = n_neighbours)

Progress :   9%|▉         | 15/169 [00:42<07:59,  3.11s/it]/home/pankil/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Progress :  14%|█▎        | 23/169 [01:06<07:06,  2.92s/it]/home/pankil/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Progress :  20%|█▉        | 33/169 [01:37<07:01,  3.10s/it]/home/pankil/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_div

{0: {'Precision Score ': 0.9427083333333333, 'Recall Score ': 0.5769230769230769, 'F1 Score ': 0.6029465930018416}, 1: {'Precision Score ': 0.9375, 'Recall Score ': 0.59375, 'F1 Score ': 0.6245614035087719}, 2: {'Precision Score ': 0.42788461538461536, 'Recall Score ': 0.489010989010989, 'F1 Score ': 0.4564102564102564}, 3: {'Precision Score ': 0.7676470588235293, 'Recall Score ': 0.711038961038961, 'F1 Score ': 0.7299382716049383}, 4: {'Precision Score ': 0.7714285714285715, 'Recall Score ': 0.7209302325581395, 'F1 Score ': 0.7414772727272728}, 5: {'Precision Score ': 0.9388888888888889, 'Recall Score ': 0.5769230769230769, 'F1 Score ': 0.6007889546351085}, 6: {'Precision Score ': 0.9484536082474226, 'Recall Score ': 0.5454545454545454, 'F1 Score ': 0.5561594202898551}, 7: {'Precision Score ': 0.7632575757575758, 'Recall Score ': 0.7632575757575758, 'F1 Score ': 0.7632575757575758}, 8: {'Precision Score ': 0.95, 'Recall Score ': 0.5833333333333334, 'F1 Score ': 0.6165413533834587}, 9:

In [54]:
f = open("binary_KNN_"+str(n_neighbours)+"_result", "wb")
pickle.dump(result_history, f)
f.close()

In [55]:
f = open("binary_KNN_"+str(n_neighbours)+"_result", "rb")
l = pickle.load(f)
f.close()

In [56]:
l = {k: v for k, v in sorted(l.items(), key=lambda item: item[1]["F1 Score "], reverse = True)}

In [57]:
# Top 20 playlists 
for key in list(l.keys())[:20]:
    print(key," = ",  l[key])

101  =  {'Precision Score ': 0.9583333333333333, 'Recall Score ': 0.9945054945054945, 'F1 Score ': 0.9754984386259908}
100  =  {'Precision Score ': 0.925, 'Recall Score ': 0.9836956521739131, 'F1 Score ': 0.9511721666417798}
72  =  {'Precision Score ': 0.9451219512195121, 'Recall Score ': 0.9451219512195121, 'F1 Score ': 0.9451219512195121}
97  =  {'Precision Score ': 0.9166666666666667, 'Recall Score ': 0.9814814814814814, 'F1 Score ': 0.9451114922813035}
137  =  {'Precision Score ': 0.9166666666666667, 'Recall Score ': 0.9775280898876404, 'F1 Score ': 0.9430512016718913}
89  =  {'Precision Score ': 0.9223985890652557, 'Recall Score ': 0.9544592030360531, 'F1 Score ': 0.936473165388828}
99  =  {'Precision Score ': 0.8936708860759495, 'Recall Score ': 0.9461979913916786, 'F1 Score ': 0.9169044821218735}
102  =  {'Precision Score ': 0.8889228886168911, 'Recall Score ': 0.9462780898876404, 'F1 Score ': 0.9142857142857144}
45  =  {'Precision Score ': 0.8938271604938272, 'Recall Score ': 0

In [58]:
f1_list = []
for key in list(l.keys()):
    f1_list.append(l[key]['F1 Score '])
print(f1_list)

[0.9754984386259908, 0.9511721666417798, 0.9451219512195121, 0.9451114922813035, 0.9430512016718913, 0.936473165388828, 0.9169044821218735, 0.9142857142857144, 0.9136904761904763, 0.9127501429388221, 0.8995073891625616, 0.8980698330080243, 0.8980698330080243, 0.892777364110201, 0.8870513402998637, 0.8844004065040652, 0.8771187589835381, 0.8722002635046113, 0.8640626724042811, 0.8624972277666889, 0.8616352201257862, 0.8585987261146497, 0.8578379521895494, 0.8577586206896552, 0.8531240918337692, 0.8471042471042471, 0.8441039307128579, 0.8410810810810811, 0.8408743169398907, 0.8387121860175752, 0.8336899158465269, 0.8332075471698114, 0.8256899771736874, 0.8230730730730731, 0.8194038573933372, 0.817502532928065, 0.8133333333333332, 0.8132884777123632, 0.8124313639358665, 0.8108255133958928, 0.8096177814707104, 0.8080143540669856, 0.8051282051282052, 0.8044170518746789, 0.8036590807675146, 0.8035597356985535, 0.8032128514056225, 0.8022598870056498, 0.8009630818619583, 0.7930529399455954, 0.

In [59]:
print(sum(f1_list)/len(f1_list))

0.7265340621265897


ANN Classification (Deep Learning)

In [91]:
result_history = run_model(model_type = "ANN")

Progress :   0%|          | 0/169 [00:00<?, ?it/s]

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_69 (Dense)             (None, 24)                840       
_________________________________________________________________
dense_70 (Dense)             (None, 12)                300       
_________________________________________________________________
dense_71 (Dense)             (None, 4)                 52        
_________________________________________________________________
dense_72 (Dense)             (None, 1)                 5         
Total params: 1,197
Trainable params: 1,197
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
7/7 [==============================] - 0s 19ms/step - loss: 0.5219 - accuracy: 0.7570 - val_loss: 0.4550 - val_accuracy: 0.8673
Epoch 2/20
7/7 [==============================] - 0s 3ms/step - loss: 0.3586 - accuracy: 0.8670 - val_loss: 0.

Progress :   1%|          | 1/169 [00:03<10:18,  3.68s/it]

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_73 (Dense)             (None, 24)                840       
_________________________________________________________________
dense_74 (Dense)             (None, 12)                300       
_________________________________________________________________
dense_75 (Dense)             (None, 4)                 52        
_________________________________________________________________
dense_76 (Dense)             (None, 1)                 5         
Total params: 1,197
Trainable params: 1,197
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
7/7 [==============================] - 0s 16ms/step - loss: 0.3924 - accuracy: 0.8505 - val_loss: 0.2880 - val_accuracy: 0.8692
Epoch 2/20
7/7 [==============================] - 0s 3ms/step - loss: 0.2292 - accuracy: 0.8995 - val_loss: 0.

Progress :   1%|          | 2/169 [00:07<10:24,  3.74s/it]

...

Model: "sequential_184"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_737 (Dense)            (None, 24)                840       
_________________________________________________________________
dense_738 (Dense)            (None, 12)                300       
_________________________________________________________________
dense_739 (Dense)            (None, 4)                 52        
_________________________________________________________________
dense_740 (Dense)            (None, 1)                 5         
Total params: 1,197
Trainable params: 1,197
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
8/8 [==============================] - 0s 27ms/step - loss: 0.5053 - accuracy: 0.7149 - val_loss: 0.4416 - val_accuracy: 0.8033
Epoch 2/20
8/8 [==============================] - 0s 3ms/step - loss: 0.3673 - accuracy: 0.8802 - val_loss: 0

Progress :  99%|█████████▉| 168/169 [11:55<00:04,  4.30s/it]

Model: "sequential_185"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_741 (Dense)            (None, 24)                840       
_________________________________________________________________
dense_742 (Dense)            (None, 12)                300       
_________________________________________________________________
dense_743 (Dense)            (None, 4)                 52        
_________________________________________________________________
dense_744 (Dense)            (None, 1)                 5         
Total params: 1,197
Trainable params: 1,197
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
7/7 [==============================] - 0s 18ms/step - loss: 0.4871 - accuracy: 0.6928 - val_loss: 0.4944 - val_accuracy: 0.7321
Epoch 2/20
7/7 [==============================] - 0s 3ms/step - loss: 0.3561 - accuracy: 0.7466 - val_loss: 0

Progress : 100%|██████████| 169/169 [11:59<00:00,  4.26s/it]

{0: {'Precision Score ': 0.8551810237203495, 'Recall Score ': 0.7574660633484163, 'F1 Score ': 0.7951933124346917}, 1: {'Precision Score ': 0.8089198036006546, 'Recall Score ': 0.7592719780219781, 'F1 Score ': 0.7806150978564772}, 2: {'Precision Score ': 0.695054945054945, 'Recall Score ': 0.8120879120879121, 'F1 Score ': 0.7228567496903813}, 3: {'Precision Score ': 0.7323131955484896, 'Recall Score ': 0.7711038961038961, 'F1 Score ': 0.7437665782493369}, 4: {'Precision Score ': 0.7648578811369509, 'Recall Score ': 0.7648578811369509, 'F1 Score ': 0.7648578811369509}, 5: {'Precision Score ': 0.8208373904576436, 'Recall Score ': 0.8208373904576436, 'F1 Score ': 0.8208373904576436}, 6: {'Precision Score ': 0.7439916405433646, 'Recall Score ': 0.7439916405433646, 'F1 Score ': 0.7439916405433646}, 7: {'Precision Score ': 0.9, 'Recall Score ': 0.9829545454545454, 'F1 Score ': 0.9357739242132306}, 8: {'Precision Score ': 0.7401433691756272, 'Recall Score ': 0.6861111111111111, 'F1 Score ': 0

In [95]:
# f = open("binary_ANN_result", "wb")
# pickle.dump(result_history, f)
# f.close()

In [96]:
f = open("binary_ANN_result", "rb")
l = pickle.load(f)
f.close()

In [97]:
l = {k: v for k, v in sorted(l.items(), key=lambda item: item[1]["F1 Score "], reverse = True)}

In [98]:
# Top 20 playlists 
for key in list(l.keys())[:20]:
    print(key," = ",  l[key])

97  =  {'Precision Score ': 0.96875, 'Recall Score ': 0.9938271604938271, 'F1 Score ': 0.9807653776798237}
101  =  {'Precision Score ': 0.9583333333333333, 'Recall Score ': 0.9945054945054945, 'F1 Score ': 0.9754984386259908}
72  =  {'Precision Score ': 0.9633126934984519, 'Recall Score ': 0.9451219512195121, 'F1 Score ': 0.9533843704995073}
7  =  {'Precision Score ': 0.9, 'Recall Score ': 0.9829545454545454, 'F1 Score ': 0.9357739242132306}
89  =  {'Precision Score ': 0.9273457466228551, 'Recall Score ': 0.944212523719165, 'F1 Score ': 0.9352678571428572}
137  =  {'Precision Score ': 0.9072295247724975, 'Recall Score ': 0.9525280898876405, 'F1 Score ': 0.9275747508305648}
103  =  {'Precision Score ': 0.8998397435897436, 'Recall Score ': 0.9455357142857144, 'F1 Score ': 0.920675105485232}
45  =  {'Precision Score ': 0.8938271604938272, 'Recall Score ': 0.9425287356321839, 'F1 Score ': 0.9136904761904763}
161  =  {'Precision Score ': 0.9050046339202966, 'Recall Score ': 0.91619047619047

In [99]:
f1_list = []
for key in list(l.keys()):
    f1_list.append(l[key]['F1 Score '])
print(f1_list)

[0.9807653776798237, 0.9754984386259908, 0.9533843704995073, 0.9357739242132306, 0.9352678571428572, 0.9275747508305648, 0.920675105485232, 0.9136904761904763, 0.9104144651872724, 0.9101181304571135, 0.9059858547524582, 0.9025000000000001, 0.9002212566310346, 0.8981481481481481, 0.8977272727272727, 0.8935952527112747, 0.8922558922558923, 0.8862797223452961, 0.8855614973262032, 0.8819627353428934, 0.8814814814814815, 0.8813140761804984, 0.8782608695652174, 0.8776455026455026, 0.8758241758241758, 0.8745718050065876, 0.8741071428571429, 0.8643264326432643, 0.8599733282530005, 0.8573529411764705, 0.8540486835568804, 0.851890756302521, 0.8514322705939472, 0.849503819518812, 0.8494485294117646, 0.8489010989010989, 0.8460031347962382, 0.8402156334231807, 0.83953216374269, 0.8385276073619632, 0.8364779874213837, 0.8346709470304976, 0.8317214700193423, 0.8315905338470005, 0.8261363636363636, 0.8256118881118881, 0.8255072130582218, 0.8252320276860154, 0.8250565062964159, 0.8208373904576436, 0.82

In [100]:
print(sum(f1_list)/len(f1_list))

0.7711823423983153
